## Imports

In [ ]:
import pandas as pd
import numpy as np
from astropy.io import fits as pyfits

import pysynphot as S # you need BT Settl models installed.

import os

## Datafiles

In [ ]:
wavelengths = np.arange(0.5, 2, 0.0001)

R_sun = 6.957e8 * 1e2 # cm
M_sun = 1.9885e30 * 1e3 # g
G = 6.67408e-11 * 1e2**3 * 1e3**-1 # m3 kg-1 s-2

stellarProps = pd.read_csv('./datafiles/stellar_props_rochester.csv')
stellarProps = stellarProps[stellarProps['Msun'].isna() == False]
stellarProps['logg'] = np.log10(G * M_sun * stellarProps['Msun'] / (R_sun * stellarProps['R_Rsun'])**2)
df_spt = stellarProps.reindex(index=stellarProps.index[::-1]).reset_index(drop='index')

## Definitions

In [ ]:
def interpolate_dfs(wavelengths, *data):
    df = pd.DataFrame({'tmp': wavelengths}, index=wavelengths)
    for dat in data:
        dat = dat[~dat.index.duplicated(keep='first')]
        df = pd.concat([df, dat], axis=1)
    df = df.interpolate('index').reindex(wavelengths)
    df = df.drop('tmp', 1)
    df = df.fillna(0)
    return df

## Code 

In [4]:
metallicity = 0
for k, star in df_spt.iterrows():
    temperature = star['Teff']
    logg = star['logg']
    
    phoenix = S.Icat('phoenix',temperature,metallicity,logg)
    phoenix.convert('Flam') # [erg/s/cm^2/A]
    phoenix.convert('Micron')
    
    # star in [J/s/m^2/μm]
    phoenix_star = pd.DataFrame({'phoenix_flux': phoenix.flux * 1e-7 * 1e4 * 1e4 }, index = phoenix.wave)
    df = interpolate_dfs(wavelengths, phoenix_star)
    df.to_csv('./datafiles/stellar_spectra/' + str(temperature) + 'K' + '.csv')

In [5]:
pwv_values = np.array([0.05, 0.1, 0.25, 0.5, 1.0, 1.5, 2.5, 3.5, 5.0, 7.5, 10.0, 20.0, 30.0])
airmass_values = np.array([1.0 , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0 , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0])

for pwv in pwv_values:
    for airmass in airmass_values:
        hdulist = pyfits.open("./datafiles/atmospheres_2400m/atm_" + str(pwv) + "_" + str(airmass) + ".fits")
        sky_df = pd.DataFrame({'sky_transmission': hdulist[1].data['trans'].byteswap().newbyteorder()}, index=hdulist[1].data['lam']/1000)
        
        df = interpolate_dfs(wavelengths, sky_df)
        df.to_csv('./datafiles/atmospheres_2400m/csv/' + str(pwv) + '_' + str(airmass) + '.csv')

In [16]:
df = pd.DataFrame({}, index=wavelengths)

rootdir = './datafiles/atmospheres_2400m/csv/'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(str(file[-4:]) == '.csv'):
            df[str(file[:-4])] = pd.read_csv(rootdir + str(file), index_col=0)
        

Rsun = 6.96342e8 # [m]
h = 6.62607004e-34
c = 299792458
rootdir = './datafiles/stellar_spectra/'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(str(file[-4:]) == '.csv'):
            R = stellarProps[stellarProps['Teff'] == int(str(file[:-5]))]['R_Rsun'].values * Rsun
            df[str(file[:-4])] = pd.read_csv(rootdir + str(file), index_col=0).apply(lambda x : x*R**2 * (x.index * 1e-6) / (h*c)) # photon/s/m2/um
        
display(df)

,0.05_1.0,0.05_1.1,0.05_1.2,0.05_1.3,0.05_1.4,0.05_1.5,0.05_1.6,0.05_1.7,0.05_1.8,0.05_1.9,...,7440K,7500K,7800K,8000K,8080K,8270K,8550K,8840K,9200K,9700K
0.5000,0.862389,0.849716,0.837229,0.824952,0.812832,0.800891,0.789126,0.777537,0.766120,0.754874,...,1.264148e+45,1.270445e+45,9.260649e+44,1.042528e+45,1.487041e+45,2.210043e+45,2.518259e+45,2.326450e+45,2.585950e+45,2.863159e+45
0.5001,0.862412,0.849740,0.837255,0.824974,0.812856,0.800916,0.789153,0.777564,0.766148,0.754903,...,1.283443e+45,1.287375e+45,9.261881e+44,1.045183e+45,1.492320e+45,2.216361e+45,2.522884e+45,2.326308e+45,2.584467e+45,2.869139e+45
0.5002,0.862434,0.849764,0.837281,0.824997,0.812879,0.800941,0.789179,0.777592,0.766177,0.754933,...,1.302746e+45,1.304311e+45,9.263112e+44,1.047839e+45,1.497601e+45,2.222681e+45,2.527511e+45,2.326165e+45,2.582982e+45,2.875121e+45
0.5003,0.862456,0.849788,0.837307,0.825019,0.812903,0.800966,0.789205,0.777619,0.766206,0.754963,...,1.322057e+45,1.321254e+45,9.264343e+44,1.050497e+45,1.502883e+45,2.229003e+45,2.532139e+45,2.326022e+45,2.581497e+45,2.881106e+45
0.5004,0.862478,0.849812,0.837333,0.825041,0.812927,0.800991,0.789232,0.777647,0.766235,0.754993,...,1.341375e+45,1.338204e+45,9.265574e+44,1.053155e+45,1.508168e+45,2.235328e+45,2.536769e+45,2.325879e+45,2.580011e+45,2.887093e+45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.9995,0.549524,0.518845,0.490100,0.509182,0.484558,0.461282,0.439277,0.418466,0.431969,0.413407,...,1.125710e+44,1.110741e+44,6.422279e+43,7.253661e+43,9.797703e+43,1.376629e+44,1.510837e+44,1.314936e+44,1.325305e+44,1.443861e+44
1.9996,0.540533,0.509484,0.480431,0.499725,0.474847,0.451361,0.429184,0.408238,0.421754,0.403077,...,1.125580e+44,1.110610e+44,6.421418e+43,7.252722e+43,9.796423e+43,1.376451e+44,1.510572e+44,1.314767e+44,1.325120e+44,1.443681e+44
1.9997,0.531543,0.500122,0.470762,0.490268,0.465136,0.441440,0.419091,0.398011,0.411539,0.392747,...,1.125451e+44,1.110480e+44,6.420556e+43,7.251784e+43,9.795142e+43,1.376273e+44,1.510308e+44,1.314598e+44,1.324935e+44,1.443502e+44
1.9998,0.522552,0.490761,0.461093,0.480810,0.455425,0.431518,0.408999,0.387783,0.401323,0.382418,...,1.125321e+44,1.110349e+44,6.419695e+43,7.250845e+43,9.793861e+43,1.376095e+44,1.510043e+44,1.314430e+44,1.324750e+44,1.443322e+44


In [8]:
df = df.interpolate(method='cubic')
df.to_pickle('./datafiles/prePWVGrid_2400m.pkl')